In [ ]:
### WARNING ###
## This notebook does not run in the same virtual enviornment
## This notenook will not run on mac 

# I attempted many pdf image extraction methods, and the only that would work with this data is Spire.
# Spire does not run on mac so I used web based jupyter notebook.
# Spire also would not let me read from web, so each pdf needed to be downloaded, runtime for me was ~20-30 minutes.

In [ ]:
print(
"""
### WARNING ###
## This notebook does not run in the same virtual enviornment
## This notenook will not run on mac 

# I attempted many pdf image extraction methods, and the only that would work with this data is Spire.
# Spire does not run on mac so I used web based jupyter notebook.
# I could not figure out how to make Spire also let me read pdf from web, so each pdf needed to be downloaded, runtime for me was ~10-30 minutes.
      

# PDF Image extraction directly implemented from Alexander Stock in Medium Article "Extract Images from PDF Documents in Python" 
# https://medium.com/@alexaae9/python-how-to-extract-images-from-pdf-documents-9492a767a613
"""
)

In [34]:
# !pip install pypdf
# !pip install Spire.Pdf

In [35]:
import pandas as pd
import numpy as np
import openpyxl
import matplotlib.pyplot as plt
import re
from pdf_scraper import run_scraper, delete_images
from tqdm import tqdm
import zipfile
import os

In [ ]:
# Read Data from Excel Spreadsheet
parcel_data = pd.read_excel('src/data/property_data/FY2026 Assessment Report (Parcel Usage)_1.xlsx')
# Find breaks in spreadsheet between 
break_points = parcel_data[parcel_data[parcel_data.columns[0]]==parcel_data[parcel_data.columns[0]].unique()[1]].index
# Loop through break points to get data
complete_df = pd.DataFrame()
for i in range(len(break_points)-1):
    df = parcel_data.loc[break_points[i]:break_points[i+1]-2]
    df_updated = pd.DataFrame(data=df.loc[break_points[i]+2:break_points[i+1]-2])
    df_updated.columns = df.loc[break_points[i]+1]
    df_updated[df.iloc[0, 0]] = [df.iloc[0,2]]*((break_points[i+1]-3)-break_points[i])
    complete_df = pd.concat([complete_df, df_updated])

## Re-label collumns and drop unnessesary ones
complete_df.columns = ['PID', 'acc_num', 'assess_dist','zoning', 
                      'owner_name', 'None', 'address', 'acres', 'business',
                      'SC_code', 'tax_land_val', 'tax_imp_val',
                      'tax_total_val', 'ex_land_val', 'ex_imp_val',
                      'ex_total_val', 'total_land_val', 'total_imp_val',
                      'total_value', 'usage']
complete_df = complete_df.drop(columns = ['acc_num', 'assess_dist', 'zoning', 'None', 'business', 'SC_code', 
                                          'tax_land_val', 'tax_imp_val', 'tax_total_val', 'ex_land_val',
                                          'ex_imp_val',	'ex_total_val',	'total_land_val', 'total_imp_val']).reset_index(drop=True)

In [37]:
## Get just Rentals
rental_df = complete_df[
complete_df['usage'].isin(['Condo (Residential) Rental', 
                           'Duplex (Town House) (Rental)', 
                           'Single-Family (Rental) ', 
                           'Single-Family (Rental) 4 Person', 
                           'Town House (Rental)', 'Apartment', 
                           'Apartment (High Rise)', 'Mixed Retail (w/Residential)',
                           'Multi-Family', 'Multi-Family (Condo)'
                          ])
]

In [ ]:
## Get Beds, Baths, Units, and Image for all rentals
## Warning: This cell will run for >10 minutes
for i in tqdm(rental_df.index, mininterval=1):
    # Run scraper for PID
    details = run_scraper(rental_df.loc[i, 'PID'])
    # Add Details to rental_df (image is saved to image folder, its path is added to the db)
    rental_df.loc[i, ['half_baths', 'full_baths', 'bedrooms', 'rental_units', 'image_path']] = details.values()
# Save CSV
rental_df.to_csv('src/data/data_cleaning/compiled_df.csv', index=False)

Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
100%|██████████| 1120/1120 [13:13<00:00,  1.41it/s]


In [ ]:
def create_png_zip(input_folder: str, output_zip: str, compress: bool = True):
    '''
    Function to ZIP all images to easily move them from jupyter to local device.
    Takes input address string, output name string, compress boolean
    Returns None.
    '''
    mode = zipfile.ZIP_DEFLATED if compress else zipfile.ZIP_STORED
    with zipfile.ZipFile(output_zip, 'w', compression=mode) as zf:
        for filename in os.listdir(input_folder):
            if filename.lower().endswith('.png'):
                file_path = os.path.join(input_folder, filename)
                try:
                    zf.write(file_path, arcname=filename)
                    os.remove(file_path)
                except Exception as e:
                    print(f"Error processing {filename}: {e}")
    print(f"Created ZIP archive: {output_zip}")
    return
input_folder = 'src/data/property_data/image_data/'
output_zip = 'src/data/proprty_data/rental_images.zip'
create_png_zip(input_folder, output_zip, compress=True)

Created ZIP archive: rental_images.zip


In [ ]:
rental_df = pd.read_csv('src/data/data_cleaning/compiled_df.csv', index_col=False)